In [2]:
import os

from dotenv import load_dotenv
from etg import GuestRoom

load_dotenv()

# ETG API Credentials
ETG_KEY_ID = os.environ["ETG_KEY_ID"]
ETG_API_KEY = os.environ["ETG_API_KEY"]
ETG_REQUEST_TIMEOUT = 30.0

SCORING_MODEL = os.environ["SCORING_MODEL"]

# Search Parameters
CITY = "Москва"

CHECKIN_DATE = "2026-03-03"
CHECKOUT_DATE = "2026-03-06"

CURRENCY = "RUB"
LANGUAGE = "ru"
RESIDENCY = "RU"

GUESTS: list[GuestRoom] = [{"adults": 2, "children": []}]
LIMIT = 800

# User preferences for AI
USER_PREFERENCES = "Чистое место не далеко от центра."

# Filters
MIN_PRICE: float | None = 5000.0  # None = no minimum
MAX_PRICE: float | None = 18000.0  # None = no maximum

# Save results to JSON
SAVE_RESULTS = True  # Set to False to disable JSON export

In [3]:
from etg import ETGClient
from utils import ostrovok_url

client = ETGClient(ETG_KEY_ID, ETG_API_KEY, timeout=ETG_REQUEST_TIMEOUT)

In [4]:
async def find_region_id(client: ETGClient, city_name: str, language: str) -> int | None:
    """Find region ID for a city name."""
    print(f"Looking up region ID for '{city_name}'...")
    regions = await client.suggest_region(city_name, language)

    if not regions:
        print(f"  No regions found for '{city_name}'")
        return None

    # Only accept City type
    for region in regions:
        if region["type"] == "City":
            region_id = region["id"]
            print(f"  Found: {region['name']} ({region.get('country_code', '')}), region_id={region_id}")
            return region_id

    # No city found - show available options
    print(f"  No city found. Available regions:")
    for region in regions[:5]:
        print(f"    - {region['name']} (type: {region['type']}, id: {region['id']})")

    return None

In [5]:
# Find region by city name.
# ETG API requires region_id for hotel search, so we first
# lookup the region ID via suggest_region by city name.
region_id = await find_region_id(client, CITY, LANGUAGE)
if not region_id:
    raise ValueError(f"Could not find region for '{CITY}'")

print(f"\nSearching hotels in {CITY}...")
print(f"  Dates: {CHECKIN_DATE} to {CHECKOUT_DATE}")
print(f"  Currency: {CURRENCY}, Limit: {LIMIT}")

Looking up region ID for 'Москва'...
  Found: Москва (RU), region_id=2395

Searching hotels in Москва...
  Dates: 2026-03-03 to 2026-03-06
  Currency: RUB, Limit: 1000


In [6]:
import pandas as pd

from services import filter_hotels_by_price, sample_hotels

# Search available hotels in the region with given parameters.
# Filters by price range if MIN_PRICE/MAX_PRICE are set.
# Returns short hotel info: id, hid, and rates (room name, price, meal).
# Full content (name, address, amenities) is fetched separately via hotel content API.
search_results = await client.search_hotels_by_region(
    region_id=region_id,
    checkin=CHECKIN_DATE,
    checkout=CHECKOUT_DATE,
    residency=RESIDENCY,
    guests=GUESTS,
    currency=CURRENCY,
    language=LANGUAGE,
    hotels_limit=LIMIT,
)

all_hotels = search_results.get("hotels", [])
total_available = search_results.get("total_hotels", len(all_hotels))

# Filter by price
filtered_hotels = filter_hotels_by_price(all_hotels, MIN_PRICE, MAX_PRICE)
total_after_filter = len(filtered_hotels)

# Sample if too many hotels (keeps processing fast and consistent with API)
sample_result = sample_hotels(filtered_hotels)
hotels = sample_result["hotels"]
sampled = sample_result["sampled"]

print(f"Total available: {total_available}")
print(f"After price filter: {total_after_filter}")
if sampled:
    print(f"Sampled down to: {sampled}")


Total available: 836
After price filter: 443


In [7]:
from services import batch_get_content

hotel_ids = [h["hid"] for h in hotels]

print(f"[batch_get_content_start] Загрузка контента для {len(hotel_ids)} отелей...")
content_map = await batch_get_content(client, hotel_ids, LANGUAGE)
print(f"[batch_get_content_done] Загружен контент для {len(content_map)} из {len(hotel_ids)} отелей")

[batch_get_content_done] Загружен контент для 443 из 443 отелей


In [8]:
from services import batch_get_reviews, filter_reviews

print(f"[batch_get_reviews_start] Загрузка отзывов для {len(hotel_ids)} отелей...")
raw_reviews_payload = await batch_get_reviews(client, hotel_ids, LANGUAGE)
reviews_map = filter_reviews(raw_reviews_payload)

total_raw = sum(rd["total_reviews"] for rd in raw_reviews_payload.values())
total_filtered = sum(len(rd["reviews"]) for rd in reviews_map.values())
# Compute filtered_by_age: how many reviews were filtered out
total_raw_in_filtered = sum(rd["total_reviews"] for rd in reviews_map.values())
total_filtered_by_age = total_raw_in_filtered - total_filtered
hotels_with_reviews = len(reviews_map)

# Calculate average rating across all hotels
all_avg_ratings = [rd["avg_rating"] for rd in reviews_map.values() if rd["avg_rating"] is not None]
overall_avg = sum(all_avg_ratings) / len(all_avg_ratings) if all_avg_ratings else 0

print(
    f"[batch_get_reviews_done] Всего {hotels_with_reviews} отелей с отзывами из {len(hotel_ids)}"
)
print(
    f"  Обработано {total_raw} отзывов → {total_filtered} релевантных "
    f"(отсечено по давности: {total_filtered_by_age})"
)
print(f"  Средний рейтинг: {overall_avg:.1f}/10")

[batch_get_reviews_done] Всего 438 отелей с отзывами из 443
  Обработано 68999 отзывов → 55028 релевантных (отсечено по давности: 13971)
  Средний рейтинг: 8.8/10


In [9]:
from services import combine_hotels_data

combined = combine_hotels_data(hotels, content_map, reviews_map)
print(f"Combined {len(combined)} hotels with content and reviews")

Combined 443 hotels with content and reviews


In [10]:
import json
from pathlib import Path

if SAVE_RESULTS:
    # Create .artifacts directory if it doesn't exist
    artifacts_dir = Path(".artifacts")
    artifacts_dir.mkdir(exist_ok=True)
    
    # Format filename: City_CheckinDate_CheckoutDate.json
    filename = f"{CITY}_{CHECKIN_DATE}_{CHECKOUT_DATE}.json"
    filepath = artifacts_dir / filename
    
    # Prepare export data with search metadata at top level
    export_data = {
        "search": {
            "city": CITY,
            "region_id": region_id,
            "checkin": CHECKIN_DATE,
            "checkout": CHECKOUT_DATE,
            "guests": GUESTS,
            "min_price": MIN_PRICE,
            "max_price": MAX_PRICE,
            "currency": CURRENCY,
            "language": LANGUAGE,
            "residency": RESIDENCY,
        },
        "hotels": combined,  # HotelFull[] - already has all data
        "stats": {
            "total_hotels": len(combined),
            "total_available": total_available,
            "total_after_filter": len(hotels),
        }
    }
    
    # Save to JSON
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(export_data, f, ensure_ascii=False, indent=2)
    
    # Format guests for display
    guests_str = "; ".join([
        f"{g['adults']} adults" + (f" + {len(g.get('children', []))} children (ages: {', '.join(map(str, g.get('children', [])))})" if g.get('children') else "")
        for g in GUESTS
    ])
    
    print(f"✅ Saved {len(combined)} hotels to {filepath}")
    print(f"   Search: {CITY}, {CHECKIN_DATE} → {CHECKOUT_DATE}")
    print(f"   Guests: {guests_str}")
    print(f"   Price: {MIN_PRICE} - {MAX_PRICE} {CURRENCY}")
else:
    print("⏭️  JSON export disabled (SAVE_RESULTS = False)")

✅ Saved 443 hotels to .artifacts/Москва_2026-03-03_2026-03-06.json
   Search: Москва, 2026-03-03 → 2026-03-06
   Guests: 2 adults
   Price: 5000.0 - 18000.0 RUB


In [11]:
import json

from services import estimate_tokens, prepare_hotel_for_llm, presort_hotels

# Parameters for review sampling
MAX_REVIEWS_PER_HOTEL = 30
REVIEW_TEXT_MAX_LENGTH = 512

# Estimate tokens before presort
hotels_for_llm_all = [
    prepare_hotel_for_llm(h, MIN_PRICE, MAX_PRICE, MAX_REVIEWS_PER_HOTEL, REVIEW_TEXT_MAX_LENGTH)
    for h in combined
]
tokens_before = estimate_tokens(json.dumps(hotels_for_llm_all, ensure_ascii=False), SCORING_MODEL)

# Pre-sort by hotel kind tier and prescore, limit to top 100 for LLM scoring
PRESORT_LIMIT = 100
top_hotels = presort_hotels(combined, reviews_map, limit=PRESORT_LIMIT)

# Estimate tokens after presort
hotels_for_llm_top = [
    prepare_hotel_for_llm(h, MIN_PRICE, MAX_PRICE, MAX_REVIEWS_PER_HOTEL, REVIEW_TEXT_MAX_LENGTH)
    for h in top_hotels
]
tokens_after = estimate_tokens(json.dumps(hotels_for_llm_top, ensure_ascii=False), SCORING_MODEL)

print(f"[presort_done] {len(combined)} отелей → {len(top_hotels)} (лимит {PRESORT_LIMIT})")
print(f"  Токены: ~{tokens_before:,} → ~{tokens_after:,} (экономия {tokens_before - tokens_after:,})")

[presort_done] 443 отелей → 100 (лимит 100)
  Токены: ~1,433,145 → ~388,264 (экономия 1,044,881)


In [12]:
import time

from services import finalize_scored_hotels, score_hotels

# Score hotels using single LLM request
print(f"[scoring_start] Scoring {len(top_hotels)} hotels...")
start_time = time.time()

scoring_result = await score_hotels(
    top_hotels,
    USER_PREFERENCES,
    guests=GUESTS,
    max_reviews=MAX_REVIEWS_PER_HOTEL,
    review_text_max_length=REVIEW_TEXT_MAX_LENGTH,
    min_price=MIN_PRICE,
    max_price=MAX_PRICE,
    currency=CURRENCY,
    top_count=10
)

elapsed = time.time() - start_time

if scoring_result["error"]:
    print(f"\n❌ ERROR: {scoring_result['error']}")
    scored_hotels = None
else:
    scoring_results = scoring_result["results"]
    print(f"[scoring_done] {len(scoring_results)} hotels scored — {elapsed:.1f}s")
    print(f"  Estimated tokens: ~{scoring_result['estimated_tokens']:,}")
    
    # Finalize scored hotels - merge scoring results with full hotel data
    scored_hotels = finalize_scored_hotels(combined, scoring_results)
    print(f"\n[finalize_done] {len(scored_hotels)} hotels with complete data")

[scoring_done] 10 hotels scored — 35.9s
  Estimated tokens: ~391,818

[finalize_done] 10 hotels with complete data


In [13]:
from services import HotelScored


def _extract_hotel_display_data(
    hotel: HotelScored,
) -> dict:
    """Extract display fields from a scored hotel."""
    hotel_id = hotel["id"]
    name = hotel["name"]
    hid = hotel["hid"]
    kind = hotel.get("kind", "")
    score = hotel["score"]
    reasons = hotel.get("top_reasons", [])
    penalties = hotel.get("score_penalties", [])
    selected_hash = hotel.get("selected_rate_hash")

    # Find selected rate by hash
    rates = hotel.get("rates", [])
    selected_rate = next((r for r in rates if r.get("match_hash") == selected_hash), None)

    # Get rate details
    if selected_rate:
        room_name = selected_rate.get("room_name", "")[:50]
        meal_data = selected_rate.get("meal_data", {})
        meal = meal_data.get("value", selected_rate.get("meal", ""))

        daily_prices = selected_rate.get("daily_prices", [])
        if daily_prices:
            total_price = sum(float(p) for p in daily_prices)
            num_nights = len(daily_prices)
            avg_price_per_night = total_price / num_nights if num_nights > 0 else 0
            pt = selected_rate.get("payment_options", {}).get("payment_types", [])
            currency = pt[0].get("show_currency_code", "") if pt else ""
            total_price_str = f"{total_price:.0f} {currency}"
            avg_price_str = f"{avg_price_per_night:.0f} {currency}"
        else:
            pt = selected_rate.get("payment_options", {}).get("payment_types", [])
            if pt:
                total_price = float(pt[0].get("show_amount", 0))
                currency = pt[0].get("show_currency_code", "")
                total_price_str = f"{total_price:.0f} {currency}"
                avg_price_str = f"{total_price:.0f} {currency}"
            else:
                total_price_str = "N/A"
                avg_price_str = "N/A"
    else:
        room_name = "N/A"
        meal = "N/A"
        total_price_str = "N/A"
        avg_price_str = "N/A"

    reviews = hotel.get("reviews")
    avg_rating = reviews.get("avg_rating") if reviews else None
    detailed = reviews.get("detailed_averages", {}) if reviews else {}

    url = ostrovok_url(
        hotel_id=hotel_id,
        hid=hid,
        checkin=CHECKIN_DATE,
        checkout=CHECKOUT_DATE,
        guests=GUESTS,
        region_id=region_id,
    )

    return {
        "hotel_id": hotel_id,
        "name": name,
        "hid": hid,
        "kind": kind,
        "score": score,
        "reasons": reasons,
        "penalties": penalties,
        "room_name": room_name,
        "meal": meal,
        "total_price_str": total_price_str,
        "avg_price_str": avg_price_str,
        "avg_rating": avg_rating,
        "detailed": detailed,
        "url": url,
    }


def print_top_hotels(
    scored_hotels: list[HotelScored],
    top_n: int = 10,
) -> None:
    """Print top N scored hotels with details and Ostrovok links."""
    print(f"\n{'='*80}")
    print(f"TOP {top_n} HOTELS")
    print(f"{'='*80}\n")

    for i, hotel in enumerate(scored_hotels[:top_n], 1):
        d = _extract_hotel_display_data(hotel)
        print(f"{i}. {d['name']} [{d['kind']}]")
        if d["avg_rating"]:
            print(f"   Score: {d['score']}/100 | Rating: {d['avg_rating']}/10")
        else:
            print(f"   Score: {d['score']}/100")
        print(f"   Room: {d['room_name']}")
        print(f"   Total: {d['total_price_str']} | Avg per night: {d['avg_price_str']} | Meal: {d['meal']}")
        if d["reasons"]:
            print(f"   + {'; '.join(d['reasons'][:3])}")
        if d["penalties"]:
            print(f"   - {'; '.join(d['penalties'][:5])}")
        print(f"   🔗 {d['url']}")
        print()

    selected = min(top_n, len(scored_hotels))
    print(f"Всего найдено {len(combined)} отелей на эти даты.")
    print(f"Подобраны лучшие {selected} по вашим критериям.")


print_top_hotels(scored_hotels, top_n=10)


TOP 10 HOTELS

1. Отель Maidens Hotel [Hotel]
   Score: 98/100 | Rating: 9.8/10
   Room: Двухместный номер Premium с видом на внутренний дв
   Total: 52302 RUB | Avg per night: 17434 RUB | Meal: nomeal
   + Exceptional cleanness score of 10.0 perfectly matches your request; Brand new 5-star hotel (built 2024) with modern amenities; Excellent location (9.8) near the city center
   - Price is at the high end of the requested range (95% position); Rating 9.8 is exceptional but not 10.0
   🔗 https://ostrovok.ru/hotel/russia/moscow/mid13100159/hotel_maidens_hotel/?dates=03.03.2026-06.03.2026&guests=2&q=2395

2. Отель V Hotel Tverskaya [Hotel]
   Score: 96/100 | Rating: 9.3/10
   Room: Двухместный номер Normal (двуспальная кровать)
   Total: 44370 RUB | Avg per night: 14790 RUB | Meal: breakfast
   + Ideal price positioning at 75% of your budget range; Exceptional location (9.8) right in the heart of Tverskaya district; High cleanness score (9.6) meets your primary preference
   - 4-star ra

In [14]:
def top_hotels_dataframe(
    scored_hotels: list[HotelScored],
    top_n: int = 10,
) -> pd.DataFrame:
    """Build a DataFrame with top N scored hotels."""
    data = []
    for hotel in scored_hotels[:top_n]:
        d = _extract_hotel_display_data(hotel)
        data.append({
            "name": d["name"][:35],
            "kind": d["kind"],
            "room": d["room_name"][:30],
            "total": d["total_price_str"],
            "avg/night": d["avg_price_str"],
            "meal": d["meal"],
            "score": d["score"],
            "rating": d["avg_rating"],
            "clean": d["detailed"].get("cleanness"),
            "url": d["url"],
        })

    df = pd.DataFrame(data)
    df.index = range(1, len(df) + 1)
    return df


pd.set_option("display.max_colwidth", 100)
top_hotels_dataframe(scored_hotels, top_n=10)

,name,kind,room,total,avg/night,meal,score,rating,clean,url
1,Отель Maidens Hotel,Hotel,Двухместный номер Premium с ви,52302 RUB,17434 RUB,nomeal,98,9.8,10.0,https://ostrovok.ru/hotel/russia/moscow/mid13100159/hotel_maidens_hotel/?dates=03.03.2026-06.03....
2,Отель V Hotel Tverskaya,Hotel,Двухместный номер Normal (двус,44370 RUB,14790 RUB,breakfast,96,9.3,9.6,https://ostrovok.ru/hotel/russia/moscow/mid9080044/vhotel_tverskaya/?dates=03.03.2026-06.03.2026...
3,Отель MYS BOUTIQUE HOTEL,Boutique_and_Design,Двухместный номер с душем Комф,47250 RUB,15750 RUB,nomeal,95,9.5,9.8,https://ostrovok.ru/hotel/russia/moscow/mid10674888/mys_boutiqueotel/?dates=03.03.2026-06.03.202...
4,Отель Soluxe Hotel Moscow,Hotel,Двухместный номер Premium (дву,41130 RUB,13710 RUB,nomeal,94,9.4,9.6,https://ostrovok.ru/hotel/russia/moscow/mid10480541/soluxe_hotel_moscow/?dates=03.03.2026-06.03....
5,Отель Савой,Hotel,Двухместный номер Classic (дву,47700 RUB,15900 RUB,nomeal,93,9.2,9.3,https://ostrovok.ru/hotel/russia/moscow/mid7573372/savoy_hotel/?dates=03.03.2026-06.03.2026&gues...
6,Отель Особняк 1830,Hotel,Двухместный номер Стандартный,36000 RUB,12000 RUB,nomeal,92,9.7,9.7,https://ostrovok.ru/hotel/russia/moscow/mid10213247/osobnyak_na_syezzhinskom_apartments_2/?dates...
7,Отель Magic Harp,Hotel,Двухместный номер Economy (дву,34352 RUB,11451 RUB,breakfast,91,9.5,9.7,https://ostrovok.ru/hotel/russia/moscow/mid11020434/magic_harp_hotel/?dates=03.03.2026-06.03.202...
8,Бутик-отель Петровка 26,Boutique_and_Design,Двухместный номер Standard (дв,43500 RUB,14500 RUB,nomeal,90,9.1,9.5,https://ostrovok.ru/hotel/russia/moscow/mid10675430/petrovka_26_boutiquehotel/?dates=03.03.2026-...
9,Отель Hyatt Regency Moscow Petrovsk,Hotel,Двухместный номер Стандартный,48000 RUB,16000 RUB,nomeal,89,9.2,9.3,https://ostrovok.ru/hotel/russia/moscow/mid10001298/petrovskij_park_congress_hotel/?dates=03.03....
10,Отель Звёзды Арбата (ранее Мариотт,Hotel,Двухместный номер Делюкс (двус,46920 RUB,15640 RUB,nomeal,88,9.2,9.2,https://ostrovok.ru/hotel/russia/moscow/mid8139930/moscow_marriott_hotel_novy_arbat/?dates=03.03...
